# Query Runner

Note: Prior to running this notebook to execute your queries, you must first run the 'etl.py' file to create
the keyspace, tables, and perform the ETL pipeline to make the data available to query.

In [1]:
# Python imports
import pandas as pd
import cassandra
import re
import numpy as np
from cassandra.cluster import Cluster

In [2]:
# Create database connection
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('sparkifydb')

In [4]:
# Import select statements from sql_queries.py
from cql_queries import(
    get_lengths_query,
    get_sessions_query,
    get_listeners_query
)

#### Query 1

#### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Table primary key is <code>PRIMARY KEY(sessionId, itemInSession)</code> to allow for filtering in <code>WHERE</code> clause on the <code>sessionId</code> partition key, and the <code>itemInSession</code> clustering column.

In [5]:
df = pd.DataFrame(list(session.execute(get_lengths_query)))
print(df)

  artist_name                       song_title    song_len
0   Faithless  Music Matters (Mark Knight Dub)  495.307312


#### Query 2

#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Table primary key is <code>PRIMARY KEY(userId, sessionId, song_name, itemInSession)</code> to allow for filtering in <code>WHERE</code> clause on the <code>userId</code> partition key, and <code>sessionId</code> clustering column, as well as sorting by the <code>iteminSession</code> clustering column.

In [6]:
df_2 = pd.DataFrame(list(session.execute(get_sessions_query)))
print(df_2)

         artist_name                                          song_name  \
0      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   
1       Three Drives                                        Greece 2000   
2   Down To The Bone                                 Keep On Keepin' On   
3  Sebastien Tellier                                          Kilometer   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


#### Query 3

#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Table primary key is <code>PRIMARY KEY(song, first_name, last_name, userId)</code> to allow for filtering in <code>WHERE</code> clause on the <code>song</code> partition key, and the clustering columns <code>first_name</code>, <code>last_name</code>, and <code>userId</code> are used to ensure each row is unique.

In [7]:
df_3 = pd.DataFrame(list(session.execute(get_listeners_query)))
print(df_3)

   first_name last_name
0  Jacqueline     Lynch
1        Sara   Johnson
2       Tegan    Levine


# Close database connection

In [8]:
session.shutdown()
cluster.shutdown()